# Task 2

Implement a random Metropolis-Hastings MCMC algorithm, based on the true likelihood (as computed by the exact method proposed by the author), to estimate the parameters of the model for the data mentioned in the paper.